### Set up the model

In [1]:
import cellbox
import os
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
import shutil
import argparse
import json
import glob
import time
from tensorflow.compat.v1.errors import OutOfRangeError
from cellbox.utils import TimeLogger
tf.disable_v2_behavior()

2023-07-11 22:59:20.780258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 22:59:20.960599: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-11 22:59:20.966947: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/lsf10/10.1/linux3.10-glibc2.17-x86_64/lib:/data/weirauchlab/opt/lib

Instructions for updating:
non-resource variables are not supported in the long term
   _____     _ _ ____              
  / ____|   | | |  _ \             
 | |     ___| | | |_) | _____  __  
 | |    / _ \ | |  _ < / _ \ \/ /  
 | |___|  __/ | | |_) | (_) >  <   
  \_____\___|_|_|____/ \___/_/\_\  
Running CellBox scripts developed in Sander lab
Maintained by Bo Yuan, Judy Shen, and Augustin Luna; contributions by Daniel Ritter

        version 0.3.2
        -- Feb 10, 2023 --
        * Modify CellBox to support TF2     
        
Tutorials and documentations are available at https://github.com/sanderlab/CellBox
If you want to discuss the usage or to report a bug, please use the 'Issues' function at GitHub.
If you find CellBox useful for your research, please consider citing the corresponding publication.
For more information, please email us at boyuan@g.harvard.edu and c_shen@g.harvard.edu, augustin_luna@hms.harvard.edu
 ----------------------------------------------------------------

In [2]:
def set_seed(in_seed):
    int_seed = int(in_seed)
    tf.compat.v1.set_random_seed(int_seed)
    np.random.seed(int_seed)


def prepare_workdir(in_cfg):
    # Read Data
    in_cfg.root_dir = os.getcwd()
    in_cfg.node_index = pd.read_csv(in_cfg.node_index_file, header=None, names=None) \
        if hasattr(in_cfg, 'node_index_file') else pd.DataFrame(np.arange(in_cfg.n_x))

    # Create Output Folder
    experiment_path = 'results/{}_{}'.format(in_cfg.experiment_id, md5)
    try:
        os.makedirs(experiment_path)
    except Exception:
        pass
    out_cfg = vars(in_cfg)
    out_cfg = {key: out_cfg[key] for key in out_cfg if type(out_cfg[key]) is not pd.DataFrame}
    os.chdir(experiment_path)
    json.dump(out_cfg, open('config.json', 'w'), indent=4)

    if "leave one out" in in_cfg.experiment_type:
        try:
            in_cfg.model_prefix = '{}_{}'.format(in_cfg.model_prefix, in_cfg.drug_index)
        except Exception('Drug index not specified') as e:
            raise e

    in_cfg.working_index = in_cfg.model_prefix + "_" + str(working_index).zfill(3)

    try:
        shutil.rmtree(in_cfg.working_index)
    except Exception:
        pass
    os.makedirs(in_cfg.working_index)
    os.chdir(in_cfg.working_index)

    with open("record_eval.csv", 'w') as f:
        f.write("epoch,iter,train_loss,valid_loss,train_mse,valid_mse,test_mse,time_elapsed\n")

    print('Working directory is ready at {}.'.format(experiment_path))
    return 0

experiment_config_path = "/users/ngun7t/Documents/cellbox-jun-6/configs_dev/Example.random_partition.CellBox.json"
working_index = 0
stage = {
    "nT": 100,
    "sub_stages":[
        {"lr_val": 0.1,"l1lambda": 0.01, "n_iter_patience":1000},
        {"lr_val": 0.01,"l1lambda": 0.01},
        {"lr_val": 0.01,"l1lambda": 0.0001},
        {"lr_val": 0.001,"l1lambda": 0.00001}
    ]}

cfg = cellbox.config.Config(experiment_config_path)
cfg.ckpt_path_full = os.path.join('./', cfg.ckpt_name)
md5 = cellbox.utils.md5(cfg)
cfg.drug_index = 5         # Change this for testing purposes
cfg.seed = working_index + cfg.seed if hasattr(cfg, "seed") else working_index + 1000
set_seed(cfg.seed)
print(vars(cfg))

prepare_workdir(cfg)
logger = cellbox.utils.TimeLogger(time_logger_step=1, hierachy=3)
args = cfg
for i, stage in enumerate(cfg.stages):
    set_seed(cfg.seed)
    cfg = cellbox.dataset.factory(cfg)
    args.sub_stages = stage['sub_stages']
    args.n_T = stage['nT']
    model = cellbox.model.factory(args)
    if i == 0: break

{'experiment_id': 'Example_RP', 'model_prefix': 'seed', 'ckpt_name': 'model11.ckpt', 'export_verbose': 3, 'experiment_type': 'random partition', 'sparse_data': False, 'batchsize': 4, 'trainset_ratio': 0.7, 'validset_ratio': 0.8, 'n_batches_eval': None, 'add_noise_level': 0, 'dT': 0.1, 'ode_solver': 'heun', 'envelope_form': 'tanh', 'envelope': 0, 'pert_form': 'by u', 'ode_degree': 1, 'ode_last_steps': 2, 'n_iter_buffer': 50, 'n_iter_patience': 100, 'weight_loss': 'None', 'l1lambda': 0.0001, 'l2lambda': 0.0001, 'model': 'CellBox', 'pert_file': '/users/ngun7t/Documents/cellbox-jun-6/data/pert.csv', 'expr_file': '/users/ngun7t/Documents/cellbox-jun-6/data/expr.csv', 'node_index_file': '/users/ngun7t/Documents/cellbox-jun-6/data/node_Index.csv', 'n_protein_nodes': 82, 'n_activity_nodes': 87, 'n_x': 99, 'envelop_form': 'tanh', 'envelop': 0, 'n_epoch': 100, 'n_iter': 100, 'stages': [{'nT': 200, 'sub_stages': [{'lr_val': 0.001, 'l1lambda': 0.0001}]}], 'ckpt_path_full': './model11.ckpt', 'drug_

In [4]:
def save_model(saver, sess, path):
    """save model"""
    # Save the variables to disk.
    tmp = saver.save(sess, path)
    print("Model saved in path: %s" % tmp)

def append_record(filename, contents):
    """define function for appending training record"""
    with open(filename, 'a') as f:
        for content in contents:
            f.write('{},'.format(content))
        f.write('\n')


def eval_model(sess, eval_iter, obj_fn, eval_dict, return_avg=True, n_batches_eval=None):
    """simulate the model for prediction"""
    sess.run(eval_iter.initializer, feed_dict=eval_dict)
    counter = 0
    eval_results = []
    while True:
        try:
            eval_results.append(sess.run(obj_fn, feed_dict=eval_dict))
        except OutOfRangeError:
            break
        counter += 1
        if n_batches_eval is not None and counter > n_batches_eval:
            break

    #print(f"eval_model eval_results: {eval_results[0].shape} with len {len(eval_results)}")
    if return_avg:
        return np.mean(np.array(eval_results), axis=0)
    return np.vstack(eval_results)

In [5]:
class Screenshot(dict):
    """summarize the model"""
    def __init__(self, args, n_iter_buffer):
        # initialize loss_min
        super().__init__()
        self.loss_min = 1000
        # initialize tuning_metric
        self.saved_losses = [self.loss_min]
        self.n_iter_buffer = n_iter_buffer
        # initialize verbose
        self.summary = {}
        self.summary = {}
        self.substage_i = []
        self.export_verbose = args.export_verbose

    def avg_n_iters_loss(self, new_loss):
        """average the last few losses"""
        self.saved_losses = self.saved_losses + [new_loss]
        self.saved_losses = self.saved_losses[-self.n_iter_buffer:]
        return sum(self.saved_losses) / len(self.saved_losses)

    def screenshot(self, sess, model, substage_i, node_index, loss_min, args):
        """evaluate models"""
        self.substage_i = substage_i
        self.loss_min = loss_min
        # Save the variables to disk.
        if self.export_verbose > 0:
            params = sess.run(model.params)
            for item in params:
                try:
                    params[item] = pd.DataFrame(params[item], index=node_index[0])
                except Exception:
                    params[item] = pd.DataFrame(params[item])
            self.update(params)

        if self.export_verbose > 1 or self.export_verbose == -1:  # no params but y_hat
            sess.run(model.iter_eval.initializer, feed_dict=model.args.feed_dicts['test_set'])
            y_hat = eval_model(sess, model.iter_eval, model.eval_yhat, args.feed_dicts['test_set'], return_avg=False)
            y_hat = pd.DataFrame(y_hat, columns=node_index[0])
            self.update({'y_hat': y_hat})

        if self.export_verbose > 2:
            try:
                # TODO: not yet support data iterators
                summary_train = sess.run(model.convergence_metric,
                                         feed_dict={model.in_pert: args.dataset['pert_train']})
                summary_test = sess.run(model.convergence_metric, feed_dict={model.in_pert: args.dataset['pert_test']})
                summary_valid = sess.run(model.convergence_metric,
                                         feed_dict={model.in_pert: args.dataset['pert_valid']})
                summary_train = pd.DataFrame(summary_train, columns=[node_index.values + '_mean', node_index.values +
                                                                     '_sd', node_index.values + '_dxdt'])
                summary_test = pd.DataFrame(summary_test, columns=[node_index.values + '_mean', node_index.values +
                                                                   '_sd', node_index.values + '_dxdt'])
                summary_valid = pd.DataFrame(summary_valid, columns=[node_index.values + '_mean', node_index.values +
                                                                     '_sd', node_index.values + '_dxdt'])
                self.update(
                    {'summary_train': summary_train, 'summary_test': summary_test, 'summary_valid': summary_valid}
                )
            except Exception:
                pass

    def save(self):
        """save model parameters"""
        for file in glob.glob(str(self.substage_i) + "_best.*.csv"):
            os.remove(file)
        for key in self:
            self[key].to_csv("{}_best.{}.loss.{}.csv".format(self.substage_i, key, self.loss_min))

In [6]:
def train_substage(model, sess, lr_val, l1_lambda, l2_lambda, n_epoch, n_iter, n_iter_buffer, n_iter_patience, args):
    """
    Training function that does one stage of training. The stage training can be repeated and modified to give better
    training result.

    Args:
        model (CellBox): an CellBox instance
        sess (tf.Session): current session, need reinitialization for every nT
        lr_val (float): learning rate (read in from config file)
        l1_lambda (float): l1 regularization weight
        l2_lambda (float): l2 regularization weight
        n_epoch (int): maximum number of epochs
        n_iter (int): maximum number of iterations
        n_iter_buffer (int): training loss moving average window
        n_iter_patience (int): training loss tolerance
        args: Args or configs
    """

    stages = glob.glob("*best*.csv")
    try:
        substage_i = 1 + max([int(stage[0]) for stage in stages])
    except Exception:
        substage_i = 1

    best_params = Screenshot(args, n_iter_buffer)

    n_unchanged = 0
    idx_iter = 0
    for key in args.feed_dicts:
        args.feed_dicts[key].update({
            model.lr: lr_val,
            model.l1_lambda: l1_lambda,
            model.l2_lambda: l2_lambda
        })
    args.logger.log("--------- lr: {}\tl1: {}\tl2: {}\t".format(lr_val, l1_lambda, l2_lambda))
    sess.run(model.iter_monitor.initializer, feed_dict=args.feed_dicts['valid_set'])
    loss_train_across_epochs, loss_train_mse_across_epochs = [], []
    loss_val_across_epochs, loss_val_mse_across_epochs = [], []
    for idx_epoch in range(n_epoch):

        loss_train_l, loss_train_mse_l = [], []
        loss_val_l, loss_val_mse_l = [], []
        if idx_iter > n_iter or n_unchanged > n_iter_patience:
            break

        sess.run(model.iter_train.initializer, feed_dict=args.feed_dicts['train_set'])
        while True:
            if idx_iter > n_iter or n_unchanged > n_iter_patience:
                break
            t0 = time.perf_counter()
            try:
                _, loss_train_i, loss_train_mse_i = sess.run(
                    (model.op_optimize, model.train_loss, model.train_mse_loss), feed_dict=args.feed_dicts['train_set'])
                #print(f"Loss train i: {loss_train_i}")
                loss_train_l.append(loss_train_i)
                loss_train_mse_l.append(loss_train_mse_i)

            except OutOfRangeError:  # for iter_train
                break

            # record training
            loss_valid_i, loss_valid_mse_i = sess.run(
                (model.monitor_loss, model.monitor_mse_loss), feed_dict=args.feed_dicts['valid_set'])
            loss_val_l.append(loss_valid_i)
            loss_val_mse_l.append(loss_valid_mse_i)
            #new_loss = best_params.avg_n_iters_loss(loss_valid_i)
            #if args.export_verbose > 0:
            #    print(("Substage:{}\tEpoch:{}/{}\tIteration: {}/{}" + "\tloss (train):{:1.6f}" + "\tbest:{:1.6f}\tTolerance: {}/{}").format(
            #        substage_i, idx_epoch, n_epoch, idx_iter,
            #        n_iter, loss_train_i,
            #        best_params.loss_min, n_unchanged,
            #        n_iter_patience
            #        ))
            new_loss = best_params.avg_n_iters_loss(loss_valid_i)
            if args.export_verbose > 0:
                print(("Substage:{}\tEpoch:{}/{}\tIteration: {}/{}" + "\tloss (train):{:1.6f}\tloss (buffer on valid):"
                       "{:1.6f}" + "\tbest:{:1.6f}\tTolerance: {}/{}").format(substage_i, idx_epoch, n_epoch, idx_iter,
                                                                              n_iter, loss_train_i, new_loss,
                                                                              best_params.loss_min, n_unchanged,
                                                                              n_iter_patience))
            append_record("record_eval.csv",
                          [idx_epoch, idx_iter, loss_train_i, loss_valid_i, loss_train_mse_i,
                           loss_valid_mse_i, None, time.perf_counter() - t0])
            # early stopping
            idx_iter += 1
            if new_loss < best_params.loss_min:
                n_unchanged = 0
                best_params.screenshot(sess, model, substage_i, args=args,
                                       node_index=args.dataset['node_index'], loss_min=new_loss)
            else:
                n_unchanged += 1

        loss_train_across_epochs.append(loss_train_l)
        loss_train_mse_across_epochs.append(loss_train_mse_l)
        loss_val_across_epochs.append(loss_val_l)
        loss_val_mse_across_epochs.append(loss_val_mse_l)


    return best_params, {
        "train": loss_train_across_epochs,
        "train_mse": loss_train_mse_across_epochs,
        "val": loss_val_across_epochs,
        "val_mse": loss_val_mse_across_epochs
    }


    # Evaluation on valid set
    #t0 = time.perf_counter()
    #sess.run(model.iter_eval.initializer, feed_dict=args.feed_dicts['valid_set'])
    #loss_valid_i, loss_valid_mse_i = eval_model(sess, model.iter_eval, (model.eval_loss, model.eval_mse_loss),
    #                                            args.feed_dicts['valid_set'], n_batches_eval=args.n_batches_eval)
    #append_record("record_eval.csv", [-1, None, None, loss_valid_i, None, loss_valid_mse_i, None, time.perf_counter() - t0])
#
    ## Evaluation on test set
    #t0 = time.perf_counter()
    #sess.run(model.iter_eval.initializer, feed_dict=args.feed_dicts['test_set'])
    #loss_test_mse = eval_model(sess, model.iter_eval, model.eval_mse_loss,
    #                           args.feed_dicts['test_set'], n_batches_eval=args.n_batches_eval)
    #append_record("record_eval.csv", [-1, None, None, None, None, None, loss_test_mse, time.perf_counter() - t0])
#
    #best_params.save()
    #args.logger.log("------------------ Substage {} finished!-------------------".format(substage_i))
    #save_model(args.saver, sess, './' + args.ckpt_name)
#
#
def append_record(filename, contents):
    """define function for appending training record"""
    with open(filename, 'a') as f:
        for content in contents:
            f.write('{},'.format(content))
        f.write('\n')

In [7]:
def train_model(model, args):
    """Train the model"""
    args.logger = TimeLogger(time_logger_step=1, hierachy=2)

    # Check if all variables in scope
    # TODO: put variables under appropriate scopes
    for i in tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope='initialization'):
        print(i)

    # Initialization
    args.saver = tf.compat.v1.train.Saver()
    from tensorflow.core.protobuf import rewriter_config_pb2
    config = tf.compat.v1.ConfigProto()
    off = rewriter_config_pb2.RewriterConfig.OFF
    config.graph_options.rewrite_options.memory_optimization = off

    # Launching session
    sess = tf.compat.v1.Session(config=config)
    sess.run(tf.compat.v1.global_variables_initializer())
    try:
        args.saver.restore(sess, './' + args.ckpt_name)
        print('Load existing model at {}...'.format(args.ckpt_name))
    except Exception:
        print('Create new model at {}...'.format(args.ckpt_name))

    # Training
    for substage in args.sub_stages:
        n_iter_buffer = substage['n_iter_buffer'] if 'n_iter_buffer' in substage else args.n_iter_buffer
        n_iter = substage['n_iter'] if 'n_iter' in substage else args.n_iter
        n_iter_patience = substage['n_iter_patience'] if 'n_iter_patience' in substage else args.n_iter_patience
        n_epoch = substage['n_epoch'] if 'n_epoch' in substage else args.n_epoch
        l1 = substage['l1lambda'] if 'l1lambda' in substage else args.l1lambda if hasattr(args, 'l1lambda') else 0
        l2 = substage['l2lambda'] if 'l2lambda' in substage else args.l2lambda if hasattr(args, 'l2lambda') else 0
        screenshot, d = train_substage(model, sess, substage['lr_val'], l1_lambda=l1, l2_lambda=l2, n_epoch=n_epoch,
                       n_iter=n_iter, n_iter_buffer=n_iter_buffer, n_iter_patience=n_iter_patience, args=args)

    # Terminate session
    sess.close()
    tf.compat.v1.reset_default_graph()

    return screenshot, d

### Train the model

In [8]:
screenshot, d = train_model(model, args)

<tf.Variable 'initialization/W:0' shape=(99, 99) dtype=float32_ref>
<tf.Variable 'initialization/eps:0' shape=(99, 1) dtype=float32_ref>
<tf.Variable 'initialization/alpha:0' shape=(99, 1) dtype=float32_ref>


2023-07-11 23:00:54.651684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 23:00:54.665376: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/lsf10/10.1/linux3.10-glibc2.17-x86_64/lib:/data/weirauchlab/opt/lib:/data/weirauchlab/opt/lib64:/data/weirauchlab/local/lib
2023-07-11 23:00:54.665446: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-11 23:00:54.665497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel dr

Create new model at model11.ckpt...
########   --------- lr: 0.001	l1: 0.0001	l2: 0.0001	   --time elapsed: 2.39
Loss train i: 2.149747133255005
Substage:1	Epoch:0/100	Iteration: 0/100	loss (train):2.149747	loss (buffer on valid):501.137357	best:1000.000000	Tolerance: 0/100
eval_model eval_results: (4, 99) with len 7
Loss train i: 2.4056596755981445
Substage:1	Epoch:0/100	Iteration: 1/100	loss (train):2.405660	loss (buffer on valid):334.835955	best:501.137357	Tolerance: 0/100
eval_model eval_results: (4, 99) with len 7
Loss train i: 2.418765068054199
Substage:1	Epoch:0/100	Iteration: 2/100	loss (train):2.418765	loss (buffer on valid):251.727617	best:334.835955	Tolerance: 0/100
eval_model eval_results: (4, 99) with len 7
Loss train i: 2.1666769981384277
Substage:1	Epoch:0/100	Iteration: 3/100	loss (train):2.166677	loss (buffer on valid):201.812260	best:251.727617	Tolerance: 0/100
eval_model eval_results: (4, 99) with len 7
Loss train i: 2.306957483291626
Substage:1	Epoch:0/100	Iteration